In [363]:
import numpy as np
import pandas as pd
from transaction_dates import *
from imiona import *
from collections import Counter
from samochody import *
from czesci import *

# Pracownicy

In [364]:
birth_dates_lims = [1960, 1995]
begining_date = "2014-01-01"
end_date = "2018-01-01"

def rind(a, b):
    return np.random.randint(a, b)

jobs_list = np.array(["menadżer", "sprzedawca", "mechanik", "pomocnik mechanika", 
                       "diagnosta samochodowy", "sprzątający", "informatyk", "księgowy"])
jobs_amount = np.array([1, rind(2, 5), rind(3, 6), rind(1, 3),
                        rind(1, 3), rind(1, 3), rind(1, 3), rind(0, 2)])
fixed_jobs = np.array([0, 1, 2, np.sum(jobs_amount[:2]), np.sum(jobs_amount[0:6])]) # zaczynamy z szefem, dwoma sprzedawcami i mechanikiem

employment_dates = random_transaction_dates(begining_date, end_date, np.sum(jobs_amount))
employment_dates[fixed_jobs] = begining_date  

wages_lims = [(60, 75), (45, 60), (45, 60), (35, 45),
              (47, 60), (35, 40), (40, 60), (40, 53)]

employees_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "PESEL",
                                     "miasto", "adres", "telefon", "email", "data_zatrudnienia",
                                     "data_zwolnienia", "stanowisko", "płaca"])
dismissal_dates = [[] for i in range(len(jobs_list))]
for (i, job) in enumerate(jobs_list):

    begin_times = ["" for j in range(jobs_amount[i])]
    for j in range(jobs_amount[i]):
        employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
        if np.random.rand() <= 0.5:
            first_name, last_name = random_name_man()
            pesel, birth_date = random_pesel_date_man(birth_dates_lims)
            sex = "M"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        else:
            first_name, last_name = random_name_woman()
            pesel, birth_date = random_pesel_date_woman(birth_dates_lims)
            sex = "K"
            city = random_city(prob=0.7)
            address = random_street()
            phone = random_phone()
            #email = random_email(first_name, last_name, birth_date[-4:])
            email = employee_email(first_name, last_name)
            employment_date = employment_dates[np.sum(jobs_amount[:i]) + j]
            wage = np.random.choice(np.arange(wages_lims[i][0], wages_lims[i][1], 0.1))
        begin_times[j] = employment_date

        employees_df.loc[np.sum(jobs_amount[:i]) + j] = [0, first_name, last_name, pesel,
                                                         city, address, phone, email, employment_date,
                                                         None, job, wage]
    dismissal_dates_j = [None for j in range(jobs_amount[i])]
    c = 0
    for j in range(len(begin_times)):
        if len(begin_times) >= 2:
            min_date = min(begin_times[:j] + begin_times[j+1:])
        else:
            min_date = begin_times[j]
        if np.random.rand() <= (jobs_amount[i]-1)/10 * 1/(1+c):
            dismissal_date = random_transaction_dates(str(max(min_date, begin_times[j])), end_date, 1)[0]
            okay = False
            for k in range(len(dismissal_dates_j)):
                if (dismissal_dates_j[k] is not None) and (dismissal_date >=  begin_times[k]) and (dismissal_date <= dismissal_dates_j[k]):
                    okay = True
                    break
                elif dismissal_date >=  begin_times[k]:
                    okay = True
                    break
            if okay:
                dismissal_dates_j[j] = dismissal_date
                c += 1
    dismissal_dates[i] = dismissal_dates_j

diss = []
for i in range(len(dismissal_dates)):
    diss += dismissal_dates[i]
employees_df["data_zwolnienia"] = diss

employees_df = employees_df.sort_values(by="data_zatrudnienia")   
employees_df["id"] =  np.arange(1, len(employees_df)+1)

In [365]:
employees_df

,id,imię,nazwisko,PESEL,miasto,adres,telefon,email,data_zatrudnienia,data_zwolnienia,stanowisko,płaca
0,1,Aleksandra,Norell,86051107828,Piła,ul. Malinowa 87/30,48008966774,aleksandra.norell@kuba.pl,2014-01-01,None,menadżer,64.3
1,2,Janusz,Kończal,64122738414,Wrocław,ul. Winna 45/11,48001146548,janusz.konczal@kuba.pl,2014-01-01,2014-03-30,sprzedawca,59.5
2,3,Dorota,Drewniak,81011601743,Wrocław,ul. Skrajna 14/6,48318168917,dorota.drewniak@kuba.pl,2014-01-01,None,sprzedawca,53.8
3,4,Józef,Stępień,69021768317,Łódź,ul. Jerzyka 84/40,48721849833,jozef.stepien@kuba.pl,2014-01-01,None,mechanik,53.0
10,5,Andrzej,Derlatka,66042806116,Olsztyn,ul. Kwiatowa 12,48145043922,andrzej.derlatka@kuba.pl,2014-01-01,None,informatyk,43.7
4,6,Wojciech,Raźny,91060884374,Wrocław,ul. Leśna 88/24,48354206498,wojciech.razny@kuba.pl,2016-01-14,2016-07-02,mechanik,59.1
5,7,Dariusz,Kuraś,91042664974,Wrocław,ul. Irysowa 44,48331844912,dariusz.kuras@kuba.pl,2016-03-05,None,mechanik,45.4
6,8,Adriana,Porwolik,82012754782,Wrocław,ul. Młyny 94/1,48866416902,adriana.porwolik@kuba.pl,2016-04-23,None,pomocnik mechanika,41.5
7,9,Piotr,Melnychuk,72041346115,Barlinek,ul. Jarzębinowa 2/20,48172252621,piotr.melnychuk@kuba.pl,2016-10-29,None,diagnosta samochodowy,49.2
8,10,Robert,Wilk,77100184413,Kalisz,ul. płk. Dyplomowanego Ludwika Bociańskiego 88/14,48047920512,robert.wilk@kuba.pl,2017-04-02,None,diagnosta samochodowy,48.7


In [366]:
employees_df.to_csv("outputs/pracownicy.csv", index=False)

# Transakcje

In [367]:
no_transactions = np.random.poisson(2000)
begining_buy = 20

dates = random_transaction_dates(begining_date, end_date, no_transactions)

In [368]:
client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions

j = 1
for i in range(len(dates)):
    prob = i/(2*len(dates))
    if np.random.rand() <= prob:
        client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
    else:
        client_ids[i] = j
        j += 1

In [369]:
car_ids = np.zeros(len(dates), dtype=int) # array with car ids for table with transactions
car_ids[:begining_buy] = np.arange(1, 21)

client_ids = np.zeros(len(dates), dtype=int) # array with client ids for table with transactions
client_ids[:begining_buy] = np.arange(1, 21)
year_ceil = [int(date[:4])-19 for date in dates[:20]]

transactions_types = np.zeros(len(dates))

clients_cars = {}
our_cars = np.arange(1, 21).tolist()
not_fixed_our_cars = our_cars[:]

jp = 21
jc = 21
for i in range(20, len(dates)):
    weights = np.array([1.5*len(clients_cars)/max(1, len(our_cars)) , len(our_cars)*50, len(clients_cars)/(max(1, len(our_cars)))])
    weights = weights/np.sum(weights)
    transaction_type = np.random.choice(np.arange(3), p=weights) # 0-buy, 1-sell, 2-service
    transactions_types[i] = transaction_type
    if transaction_type == 0:
        prob = i/(2*len(dates)) 
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars and np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                clients_cars[client_ids[i]].remove(car_ids[i])
            else:
                car_ids[i] = jc
                jc += 1
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            our_cars.append(car_ids[i])
            not_fixed_our_cars.append(car_ids[i])
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

    if transaction_type == 1:
        prob = i/(2*len(dates))
        if np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            car_ids[i] = np.random.choice(our_cars)
            if client_ids[i] in clients_cars:
                clients_cars[client_ids[i]].append(car_ids[i])
            else:
                clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
        else:
            client_ids[i] = jp
            car_ids[i] = np.random.choice(our_cars)
            clients_cars[client_ids[i]] = [car_ids[i]]
            our_cars.remove(car_ids[i])
            if car_ids[i] in not_fixed_our_cars:
                not_fixed_our_cars.remove(car_ids[i])
            jp += 1
            year_ceil.append(int(dates[i][:4])-19)
    
    if transaction_type == 2:
        prob = i/(2*len(dates))
        if len(not_fixed_our_cars) >= 1 and np.random.rand() <= weights[1]: #fixing car we currently have
           client_ids[i] = 0
           car_ids[i] = np.random.choice(not_fixed_our_cars)
           not_fixed_our_cars.remove(car_ids[i])
        elif np.random.rand() <= prob: #client that already is in database
            client_ids[i] = np.random.choice(np.unique(client_ids[:i]))
            if client_ids[i] in clients_cars:
                if np.random.rand() <= 0.6 and len(clients_cars[client_ids[i]]) >= 1:
                    car_ids[i] = np.random.choice(clients_cars[client_ids[i]])
                else:
                    car_ids[i] = jc
                    clients_cars[client_ids[i]].append(car_ids[i])
                    jc += 1
            else:
                car_ids[i] = jc
                clients_cars[client_ids[i]] = [car_ids[i]]
                jc += 1
        else:
            client_ids[i] = jp
            car_ids[i] = jc
            clients_cars[client_ids[i]] = [car_ids[i]]
            jp += 1
            jc += 1
            year_ceil.append(int(dates[i][:4])-19)

In [370]:
len(year_ceil), len(pd.unique(client_ids))

(1362, 1363)

In [371]:
year_ceil

[1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,
 1995,

In [372]:
buy_indexes = np.where(transactions_types == 0)
sell_indexes = np.where(transactions_types == 1)
service_indexes = np.where(transactions_types == 2)

In [373]:
len(buy_indexes[0]), len(sell_indexes[0]), len(service_indexes[0])

(777, 773, 441)

In [374]:
transaction_dates_buy = dates[buy_indexes]
transaction_dates_sell = dates[sell_indexes]
transaction_dates_service = dates[service_indexes]

client_ids_buy = client_ids[buy_indexes]
client_ids_sell = client_ids[sell_indexes]
client_ids_service = client_ids[service_indexes]

car_ids_buy = car_ids[buy_indexes]
car_ids_sell = car_ids[sell_indexes]
car_ids_service = car_ids[service_indexes]

In [375]:
buy_employee_id = np.zeros(len(transaction_dates_buy))

for i in range(len(transaction_dates_buy)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_buy[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_buy[i]
    buy_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [376]:
sell_employee_id = np.zeros(len(transaction_dates_sell))

for i in range(len(transaction_dates_sell)):
    filter1 = employees_df["stanowisko"] == "sprzedawca"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_sell[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_sell[i]
    sell_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])

In [377]:
service_employee_id = np.zeros(len(transaction_dates_service))

for i in range(len(transaction_dates_service)):
    filter1 = employees_df["stanowisko"] == "mechanik"
    filter2 = employees_df["data_zatrudnienia"] <= transaction_dates_service[i]
    filter3 = pd.notna(employees_df["data_zwolnienia"])
    filter4 = employees_df["data_zwolnienia"] < transaction_dates_service[i]
    service_employee_id[i] = np.random.choice(employees_df['id'][filter1 & filter2 & ~(filter3 & filter4)])


# klienci

In [378]:
year_floor = 1960
transaction_counter = Counter(client_ids)

clients_df = pd.DataFrame(columns=["id", "imię", "nazwisko", "płeć", "PESEL", "miasto", "adres", "karta_lojalnościowa", "telefon", "email"])

menager = employees_df.iloc[0]
if int(menager["PESEL"][-2]) % 2 == 1:
    menager_sex = "M"
else:
    menager_sex = "K"
clients_df.loc[0] = [0, menager["imię"], menager["nazwisko"], menager_sex, menager["PESEL"], menager["miasto"], menager["adres"], False, menager["telefon"], menager["email"]]

unique_clients = np.unique(client_ids)
unique_clients = unique_clients[unique_clients != 0]
for i in range(len(unique_clients)):
    if np.random.rand() <= 0.5: # chłop probability
        id_ = unique_clients[i]
        first_name, last_name = random_name_man()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'M'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
    else:
        id_ = unique_clients[i]
        first_name, last_name = random_name_woman()
        if id_ in client_ids_buy or id_ in client_ids_sell:
            pesel, birth_date = random_pesel_date_man([year_floor, year_ceil[i]])
            year = birth_date[-4:]
        else:
            pesel = None
            birth_date = None
            year = str(np.random.randint(year_floor, year_ceil[i]))
        sex = 'K'
        city = random_city()
        address = random_street()
        if transaction_counter[id_] >= 3:
            loyalty_card = True
        else:
            loyalty_card = False
        phone = random_phone()
        email = random_email(first_name, last_name, year)
        clients_df.loc[i+1] = [unique_clients[i], first_name, last_name, sex, pesel, city, address, loyalty_card, phone, email]
                             
clients_df.astype({"PESEL": str})

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Aleksandra,Norell,K,86051107828,Piła,ul. Malinowa 87/30,False,48008966774,aleksandra.norell@kuba.pl
1,1,Magdalena,Szołdra,K,67011084513,Gorzów Wielkopolski,ul. Spichrzowa 97,False,48941105014,n7wf10fk5x@opayq.com
2,2,Michael,Kaczmarczyk,M,71112267216,Kraków,ul. Spacerowa 63,True,48204076024,kaczmarczyk19716@wp.pl
3,3,Gerardo,Kaniuk,M,61061611835,Siemianowice Śląskie,ul. Stefana Okrzei 28,False,48248486000,kaniuk8@outlook.com
4,4,Julianna,Muda,K,91012152537,Wrocław,ul. Jana III Sobieskiego 1/22,False,48810224727,jmuda@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1358,1358,Dawid,Babicz,M,66012349755,Opole,ul. Podgórna 28/13,False,48982877124,daw_babic1966@gmail.com
1359,1359,Mykola,Nytko,M,85042114997,Osiek,ul. Krawiecka 5/39,False,48732419145,nytko19854@gmail.com
1360,1360,Natalia,Bobrowska,K,84090585294,Wrocław,ul. Plac Kazimierza Wielkiego 62/13,False,48113195797,n-bobrowska@onet.pl
1361,1361,Zdzisław,Falczyński,M,None,Bydgoszcz,ul. Wrzosowa 84/24,False,48684019860,gtzmd596l71@gmail.com


In [379]:
clients_with_lc = clients_df[clients_df["karta_lojalnościowa"]]["id"].values

loyalty_card_dates = [() for i in range(len(clients_with_lc))]
for (i, client) in enumerate(clients_with_lc):
    loyalty_card_dates[i] = client, dates[np.where(client_ids == client)[0][2]]


In [380]:
clients_df

,id,imię,nazwisko,płeć,PESEL,miasto,adres,karta_lojalnościowa,telefon,email
0,0,Aleksandra,Norell,K,86051107828,Piła,ul. Malinowa 87/30,False,48008966774,aleksandra.norell@kuba.pl
1,1,Magdalena,Szołdra,K,67011084513,Gorzów Wielkopolski,ul. Spichrzowa 97,False,48941105014,n7wf10fk5x@opayq.com
2,2,Michael,Kaczmarczyk,M,71112267216,Kraków,ul. Spacerowa 63,True,48204076024,kaczmarczyk19716@wp.pl
3,3,Gerardo,Kaniuk,M,61061611835,Siemianowice Śląskie,ul. Stefana Okrzei 28,False,48248486000,kaniuk8@outlook.com
4,4,Julianna,Muda,K,91012152537,Wrocław,ul. Jana III Sobieskiego 1/22,False,48810224727,jmuda@o2.pl
...,...,...,...,...,...,...,...,...,...,...
1358,1358,Dawid,Babicz,M,66012349755,Opole,ul. Podgórna 28/13,False,48982877124,daw_babic1966@gmail.com
1359,1359,Mykola,Nytko,M,85042114997,Osiek,ul. Krawiecka 5/39,False,48732419145,nytko19854@gmail.com
1360,1360,Natalia,Bobrowska,K,84090585294,Wrocław,ul. Plac Kazimierza Wielkiego 62/13,False,48113195797,n-bobrowska@onet.pl
1361,1361,Zdzisław,Falczyński,M,None,Bydgoszcz,ul. Wrzosowa 84/24,False,48684019860,gtzmd596l71@gmail.com


In [381]:
clients_df.to_csv("outputs/klienci.csv", index=False)

In [382]:
np.sum(clients_df['karta_lojalnościowa'])

np.int64(88)

In [383]:
np.sum(pd.isna(clients_df["PESEL"]))

np.int64(168)

# auta

In [384]:
rng = np.random.default_rng()

data_car, data_bike = get_car_data()
make_list_car, model_list_car, car_weights = get_car_weights()
make_list_bike, model_list_bike, bike_weights = get_bike_weights()

In [385]:
def create_car(i):

    is_car = rng.random() < 0.9
    colors = np.array(('czarny', 'biały', 'szary', 'srebrny', 'niebieski', 'czerwony', 'brązowy', 'złoty', 'żółty', 'inny'))
    colors_weights = np.array((0.22, 0.19, 0.18, 0.15, 0.10, 0.09, 0.03, 0.02, 0.01, 0.01))

    if is_car:
        index = rng.choice(len(make_list_car), p=car_weights)
        make, model = make_list_car[index], model_list_car[index]
        weights = data_car[(data_car['make'] == make) & (data_car['model'] == model)]
        body = np.array(('SUV', 'Hatchback', 'Sedan', 'Kombi', 'Coupe', 'Van', 'Minivan', 'Kabriolet'))
        body_weights = np.array((0.25, 0.25, 0.23, 0.17, 0.06, 0.03, 0.009, 0.001))
    else:
        index = rng.choice(len(make_list_bike), p=bike_weights)
        make, model = make_list_bike[index], model_list_bike[index]
        weights = False
        body = False
        body_weights = False

    wheels = 4 if is_car else 2
    year = get_year()
    gear = get_gear(weights, is_car)
    fuel = get_fuel(weights, is_car)
    post_accident = get_post_accident()
    hp = get_hp(weights, is_car)
    color = get_color(colors, colors_weights)
    body_type = get_body_type(body, body_weights, is_car)
    engine_capacity = get_engine_capacity(hp, is_car)
    doors = get_doors(is_car)
    seats = get_num_of_seats(is_car)
    available = i+1 in our_cars
    
    return [i+1, make, model, wheels, year, body_type, post_accident, color, fuel, engine_capacity, hp, gear, doors, seats, available]


def create_car_table(n):
    
    df = pd.DataFrame(columns=['id', 'marka', 'model', 'liczba_kół', 'rok_produkcji', 'typ_nadwozia', 'czy_powypadkowy', 'kolor', 'typ_paliwa', 'pojemność_silnika', 'moc', 'skrzynia_biegów', 'liczba_drzwi', 'liczba_miejsc', 'czy_dostępny'])
    for i in range(n):
        row = create_car(i)
        df.loc[i] = row
    return df


In [386]:
df_samochody = create_car_table(max(car_ids))

In [387]:
df_samochody

,id,marka,model,liczba_kół,rok_produkcji,typ_nadwozia,czy_powypadkowy,kolor,typ_paliwa,pojemność_silnika,moc,skrzynia_biegów,liczba_drzwi,liczba_miejsc,czy_dostępny
0,1,Opel,Insignia,4,2000,Coupe,False,niebieski,Diesel,2.2,140,Manualna,5,2,False
1,2,Opel,Corsa,4,2001,Hatchback,False,niebieski,Benzyna,1.4,90,Manualna,5,5,False
2,3,Audi,Q3,4,1996,Hatchback,False,czarny,Diesel,2.5,170,Automatyczna,5,5,False
3,4,Suzuki,Celerio,4,2003,Kombi,True,srebrny,Benzyna,1.0,68,Manualna,5,5,False
4,5,Renault,Captur,4,1995,SUV,False,szary,Diesel,2.5,154,Automatyczna,5,2,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
972,973,Ford,Kuga,4,2015,Sedan,False,niebieski,Benzyna,2.3,150,Automatyczna,5,5,False
973,974,Opel,Astra,4,2008,Hatchback,False,srebrny,Benzyna,1.5,101,Manualna,5,2,False
974,975,Peugeot,208,4,1999,Kombi,False,czarny,Benzyna,1.2,82,Manualna,5,5,True
975,976,Fiat,Freemont,4,2005,SUV,False,szary,Diesel,2.6,170,Manualna,5,5,False


In [388]:
df_samochody.to_csv("outputs/samochody.csv", index=False)

In [389]:
pd.unique(df_samochody["skrzynia_biegów"])

array(['Manualna', 'Automatyczna', 'Półautomatyczna'], dtype=object)

# Usługi

In [390]:
service_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'mechanik']['id'], len(car_ids_service))
service_ids = np.arange(len(transaction_dates_service)) + 1

df_services = pd.DataFrame()
df_services['id'] = service_ids
df_services['id_samochodu'] = car_ids_service
df_services['id_klienta'] = client_ids_service
df_services['id_pracownika'] = service_employee
df_services['data_zaksięgowania'] = transaction_dates_service

# Zakup samochodów

In [391]:
def car_purchase_price(hp, yr_bought, yr_prod, post_acc):
    return np.round(hp * 1000 / np.sqrt(int(yr_bought) - int(yr_prod) + 1) * (0.75 if post_acc else 1) * rng.uniform(0.8, 1.2), -2)

In [392]:
buy_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_buy))
buy_ids = np.arange(len(transaction_dates_buy)) + 1

df_buy = pd.DataFrame()
df_buy['id'] = buy_ids
df_buy['id_samochodu'] = car_ids_buy
df_buy['id_klienta'] = client_ids_buy
df_buy['id_pracownika'] = buy_employee
df_buy['data_zaksięgowania'] = transaction_dates_buy

In [393]:

service_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_services[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
service_date_fix['data_zaksięgowania'] = service_date_fix['data_zaksięgowania'].str[:4]

buy_date_fix = pd.merge(df_samochody[['id', 'rok_produkcji']], df_buy[['id_samochodu', 'data_zaksięgowania']], how='right', left_on='id', right_on='id_samochodu')
buy_date_fix['data_zaksięgowania'] = buy_date_fix['data_zaksięgowania'].str[:4]

for i in range(len(buy_date_fix)):
    if str(buy_date_fix['rok_produkcji'].loc()[i]) >= str(buy_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = buy_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

for i in range(len(service_date_fix)):
    if str(service_date_fix['rok_produkcji'].loc()[i]) >= str(service_date_fix['data_zaksięgowania'].loc()[i]):
        car_index = service_date_fix.loc[i, 'id_samochodu']
        df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\3526569416.py:10: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '2006' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_samochody.loc[car_index-1, 'rok_produkcji'] = str(int(df_samochody.loc[car_index-1, 'rok_produkcji']) - 10)


In [394]:
purchase_prices = []
for j, i in enumerate(car_ids_buy):
    row = df_samochody.loc()[i-1]
    year_bought = transaction_dates_buy[j][:4]
    purchase_prices.append(car_purchase_price(row['moc'], year_bought, row['rok_produkcji'], row['czy_powypadkowy']))

df_buy['cena'] = purchase_prices

In [395]:
df_buy

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,1,1,2,2014-01-02,38100.0
1,2,2,2,2,2014-01-02,23800.0
2,3,3,3,2,2014-01-02,40900.0
3,4,4,4,2,2014-01-03,17600.0
4,5,5,5,3,2014-01-03,36800.0
...,...,...,...,...,...,...
772,773,975,926,3,2017-12-23,19100.0
773,774,931,186,3,2017-12-23,27600.0
774,775,122,204,2,2017-12-26,25400.0
775,776,767,542,3,2017-12-27,23200.0


In [396]:
df_buy.to_csv("outputs/zakup_samochodu.csv", index=False)

# Sprzedaż samochodów

In [397]:
sell_employee = rng.choice(employees_df[employees_df['stanowisko'] == 'sprzedawca']['id'], len(car_ids_sell))
sell_ids = np.arange(len(transaction_dates_sell)) + 1

df_sell = pd.DataFrame()
df_sell['id'] = sell_ids
df_sell['id_samochodu'] = car_ids_sell
df_sell['id_klienta'] = client_ids_sell
df_sell['id_pracownika'] = sell_employee
df_sell['data_zaksięgowania'] = transaction_dates_sell

In [398]:
loyalty_card_customer_ids = [el[0] for el in loyalty_card_dates]
loyalty_card_dates_sep = [el[1] for el in loyalty_card_dates]

In [399]:
sell_prices = []
for i in car_ids_sell:
    b_price = df_buy[df_buy['id_samochodu'] == i]['cena'].values[0]
    sell_prices.append(np.round(b_price * rng.uniform(1.2, 1.5), -2))

for i in range(len(sell_ids)):
    customer_id = df_sell.loc()[i]['id_klienta']
    if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:
        date_index = loyalty_card_customer_ids.index(customer_id)
        if loyalty_card_dates_sep[date_index] < df_sell.loc()[i]['data_zaksięgowania']:
            sell_prices[i] *= 0.97
            #print(i)

sell_installments = []
for i in range(len(sell_ids)):
    if sell_prices[i] > 80000:
        inst_prob = [0.1, 0.2, 0.3, 0.4]
    else:
        inst_prob = [0.4, 0.3, 0.2, 0.1]
    installments = rng.choice([1, 6, 12, 18], p=inst_prob)
    if installments == 6:
        sell_prices[i] *= 1.02
    elif installments == 12:
        sell_prices[i] *= 1.04
    elif installments == 18:
        sell_prices[i] *= 1.06
    sell_installments.append(installments)
#print(sell_installments)

df_sell['cena'] = sell_prices
df_sell['liczba_rat'] = sell_installments
df_sell = df_sell[['id', 'id_samochodu', 'id_klienta', 'id_pracownika', 'cena', 'liczba_rat', 'data_zaksięgowania']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\1822833189.py:8: FutureWarning: Series.bool is now deprecated and will be removed in future version of pandas
  if clients_df[clients_df['id'] == customer_id]['karta_lojalnościowa'].bool() and customer_id in loyalty_card_customer_ids:


In [400]:
df_sell

,id,id_samochodu,id_klienta,id_pracownika,cena,liczba_rat,data_zaksięgowania
0,1,7,21,2,203528.0,12,2014-01-10
1,2,10,22,3,32200.0,1,2014-01-11
2,3,17,23,2,55300.0,1,2014-01-11
3,4,9,24,3,40188.0,6,2014-01-11
4,5,14,25,2,28050.0,6,2014-01-11
...,...,...,...,...,...,...,...
768,769,969,1359,2,235144.0,12,2017-12-28
769,770,972,1360,2,124280.0,12,2017-12-28
770,771,122,1139,2,35152.0,12,2017-12-29
771,772,974,881,2,45000.0,1,2017-12-31


In [401]:
np.sum(df_sell["cena"] == float('inf')) # wychodzą inf wiec trzeba poprawić

np.int64(0)

In [402]:
df_sell.to_csv("outputs/sprzedaż_samochodu.csv", index=False)

# Użyte części

In [403]:
data_parts = get_data_parts()
data_parts_filtered = data_parts[data_parts['motor'] == True]

def get_parts_indexes(car_id):
    if df_samochody.loc[car_id-1].values[3] == 2:
        parts_set = data_parts_filtered[['id','część']]
    else:
        parts_set = data_parts[['id','część']]
    
    parts_indexes = parts_set['id']
    num_parts = rng.choice([1, 2, 3], p=[0.6, 0.3, 0.1])
    indexes = rng.choice(parts_indexes, size=num_parts)
    return indexes

all_car_parts_indexes = [(i, get_parts_indexes(i)) for i in car_ids_service]
print(all_car_parts_indexes)

[(np.int64(21), array([52, 61])), (np.int64(22), array([43])), (np.int64(25), array([46])), (np.int64(27), array([34, 55, 25])), (np.int64(29), array([ 8, 15])), (np.int64(32), array([36])), (np.int64(36), array([59])), (np.int64(37), array([62, 21])), (np.int64(41), array([36])), (np.int64(42), array([12])), (np.int64(45), array([4])), (np.int64(48), array([25])), (np.int64(53), array([27, 39])), (np.int64(52), array([43])), (np.int64(54), array([62])), (np.int64(58), array([31])), (np.int64(59), array([27, 49])), (np.int64(60), array([10, 39])), (np.int64(66), array([40])), (np.int64(68), array([0])), (np.int64(67), array([31, 62])), (np.int64(73), array([23])), (np.int64(72), array([47])), (np.int64(74), array([37,  5])), (np.int64(76), array([40, 24])), (np.int64(79), array([60, 12])), (np.int64(83), array([23])), (np.int64(47), array([44])), (np.int64(87), array([53])), (np.int64(14), array([51])), (np.int64(88), array([63])), (np.int64(89), array([63])), (np.int64(99), array([27]

In [404]:
#df_samochody.loc()[1] - samochód o indeksie 0

def create_part(id, service_id, car):
    part = data_parts.loc()[id]
    if type(part['atrybuty']) != float:
        attributes = part['atrybuty'].split()
        for i in range(len(attributes)):
            if attributes[i] == 'marka':
                attributes[i] = car['marka']
            if attributes[i] == 'model':
                attributes[i] = car['model']
            if attributes[i] == 'kolor':
                attributes[i] = car['kolor']
            if attributes[i] == 'rozmiar':
                attributes[i] = 'R' + str(rng.choice([15, 16, 17]))
            if attributes[i] == 'material':
                attributes[i] = rng.choice(['skóra', 'materiał'], p=[0.2, 0.8])
            if attributes[i] == 'pojemnosc':
                attributes[i] = str(car['pojemność_silnika'])
            if attributes[i] == 'skrzynia':
                attributes[i] = str(car['skrzynia_biegów'])
        price = int(np.floor(part['cena']*car['pojemność_silnika']/1.8 * rng.uniform(0.8, 1.2)))
    else:
        attributes = ''
        price = part['cena']
    quantity = part['maxcount']
    return part['część'], ' '.join(attributes), price, service_id, quantity

parts = []
for service_id, element in enumerate(all_car_parts_indexes):
    car_id, parts_id = element
    car = df_samochody.loc()[car_id-1]
    for part_id in parts_id:
        parts.append(create_part(part_id, service_id+1, car))

parts = np.array(parts)
parts_df = pd.DataFrame(parts, columns=['nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']).drop_duplicates(['nazwa', 'atrybuty'])
parts_df['id'] = np.arange(1, len(parts_df)+1)
parts_df = parts_df[['id', 'nazwa', 'atrybuty', 'cena', 'id_usługi', 'ilość']]

parts = pd.DataFrame(parts, columns=['nazwa', 'atrybut', 'cena', 'id_usługi', 'ilość'])

In [405]:
parts

,nazwa,atrybut,cena,id_usługi,ilość
0,Zbiornik paliwa,Ford EcoSport,449,1,1
1,Lusterko zewnętrzne lewe,Ford EcoSport czerwony,32,1,1
2,Miska olejowa,Audi A6,1276,2,1
3,Tłumik,,200,3,1
4,Fotel przedni,Kia Venga materiał,220,4,1
...,...,...,...,...,...
669,Opona letnia,R16,321,440,4
670,Amortyzator,,150,440,4
671,Drążek kierowniczy,,100,441,1
672,Hamulec ręczny,Volkswagen T6 Multivan,90,441,1


# Wyposażenie

In [406]:
eq_len = len(parts_df)
equipment_id = parts_df['id']
equipment_name = parts_df['nazwa']
equipment_attr = parts_df['atrybuty']
equipment_price = parts_df['cena']

df_equipment = pd.DataFrame()
df_equipment['id'] = equipment_id
df_equipment['nazwa'] = equipment_name
df_equipment['atrybut'] = equipment_attr
df_equipment['cena_części'] = equipment_price.astype(float)

In [407]:
df_equipment

,id,nazwa,atrybut,cena_części
0,1,Zbiornik paliwa,Ford EcoSport,449.0
1,2,Lusterko zewnętrzne lewe,Ford EcoSport czerwony,32.0
2,3,Miska olejowa,Audi A6,1276.0
3,4,Tłumik,,200.0
4,5,Fotel przedni,Kia Venga materiał,220.0
...,...,...,...,...
665,504,Filtr paliwa,Opel Corsa,375.0
667,505,Ogniwo paliwowe,Fiat Freemont,386.0
668,506,Światło mgłowe,Opel Astra,35.0
672,507,Hamulec ręczny,Volkswagen T6 Multivan,90.0


# Użyte części - uzupełnienie

In [408]:
used_parts_df = parts.merge(df_equipment, on=['nazwa', 'atrybut'])
used_parts_df.rename(columns={'id': 'id_części'}, inplace=True)
used_parts_df['ilość'] = pd.to_numeric(used_parts_df['ilość'])
ids = np.arange(len(used_parts_df))+1
used_parts_df['id'] = ids
used_parts_df['cena'] = pd.to_numeric(used_parts_df['cena'])

for i in range(len(used_parts_df)):
    used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)
    used_parts_df.loc()[i, 'cena'] = np.round(used_parts_df.loc()[i, 'cena'], 0)

used_parts_df = used_parts_df[['id', 'id_usługi', 'id_części', 'ilość', 'cena']]

C:\Users\oskar.LAPTOP-KARLNF81\AppData\Local\Temp\ipykernel_14068\1878107147.py:9: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '428.30975800562373' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  used_parts_df.loc()[i, 'cena'] *= rng.uniform(0.9, 1.1)


In [409]:
used_parts_df

,id,id_usługi,id_części,ilość,cena
0,1,1,1,1,428.0
1,2,1,2,1,34.0
2,3,2,3,1,1382.0
3,4,3,4,1,203.0
4,5,4,5,1,204.0
...,...,...,...,...,...
669,670,440,74,4,320.0
670,671,440,53,4,135.0
671,672,441,60,1,95.0
672,673,441,507,1,86.0


In [410]:
used_parts_df.to_csv("outputs/użyte_części.csv", index=False)

# Zakup części

In [411]:
# def get_service_date(part_id):
    
#     index_used_part = used_parts_df[used_parts_df['id_części'] == part_id].index[0]
#     service_date = df_services[df_services['id'] == index_used_part]['data_zaksięgowania']
#     return service_date
    

# buy_parts_quantity = used_parts_df.groupby(['id_części'])['ilość'].sum().reset_index()
# parts_purchase_pd = pd.DataFrame(columns=['id', 'id_części', 'koszt', 'ilość'])

# for i in range(len(buy_parts_quantity)):
#     part_id = buy_parts_quantity['id_części'][i]
#     quantity = buy_parts_quantity['ilość'][i]
#     price = df_equipment['cena_części'][df_equipment['id'] == part_id].to_numpy()
#     total_cost = np.floor(price * rng.uniform(0.75, 0.95))[0]
#     service_date = get_service_date(part_id)
    
#     row = [int(i+1), int(part_id), total_cost, int(quantity)]
#     parts_purchase_pd.loc[i] = row


In [412]:
# parts_purchase_pd

In [413]:
# parts_purchase_pd.to_csv("outputs/zakup_części.csv", index=False)

In [414]:
df_equipment = df_equipment[['id', 'nazwa', 'atrybut']]

In [415]:
df_equipment.to_csv("outputs/wyposażenie.csv", index=False)

# Usługi uzupełnienie

In [416]:
service_prices = np.zeros(len(service_ids))
for i in range(len(parts)):
    row = parts.loc()[i]
    service_index = int(row['id_usługi'])
    service_prices[service_index-1] += float(row['cena']) * int(row['ilość'])

for i in range(len(df_services)):
    if df_services.loc()[i]['id_klienta'] == 0:
        df_services.loc[i, 'cena'] = 0
    else:
        df_services.loc[i, 'cena'] = np.round(service_prices[i] * (rng.uniform(1.25, 1.7)), -1).astype(int)

In [417]:
df_services

,id,id_samochodu,id_klienta,id_pracownika,data_zaksięgowania,cena
0,1,21,40,6,2014-01-25,740.0
1,2,22,42,4,2014-01-28,1780.0
2,3,25,0,7,2014-02-01,0.0
3,4,27,51,6,2014-02-04,540.0
4,5,29,53,4,2014-02-05,110.0
...,...,...,...,...,...,...
436,437,122,0,7,2017-12-27,0.0
437,438,976,1361,6,2017-12-28,540.0
438,439,974,0,7,2017-12-29,0.0
439,440,973,0,7,2017-12-29,0.0


In [418]:
df_services.to_csv("outputs/usługi.csv", index=False)